<a href="https://colab.research.google.com/github/KomalT2003/DiabeticFootDetection/blob/main/DiabeticFootDataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install Tensorflow and other libraries

In [ ]:
# step 1: install tensorflow and other libraries
!pip install tensorflow

In [ ]:
# STEP 2: Import Libraries
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
from sklearn.utils import class_weight
from google.colab import drive


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
# Define paths to training and validation datasets
base_path = '/content/drive/MyDrive/Dataset'
train_path = os.path.join(base_path, 'Train')
val_path = os.path.join(base_path, 'Validation')

# Check if the directories exist
if not os.path.exists(train_path):
    raise FileNotFoundError(f"Training directory does not exist: {train_path}")
if not os.path.exists(val_path):
    raise FileNotFoundError(f"Validation directory does not exist: {val_path}")


In [ ]:
def check_directory_contents(base_path):
    classes = ['Normal', 'Controlled', 'DFC']
    orientations = ['Front', 'Back', 'Nailbed']

    for class_name in classes:
        for orientation in orientations:
            path = os.path.join(base_path, class_name, orientation)
            if os.path.exists(path):
                num_images = len(os.listdir(path))
                print(f"Found {num_images} images in: {path}")
            else:
                print(f"Directory does not exist: {path}")

print("Checking training directories:")
check_directory_contents(train_path)

print("\nChecking validation directories:")
check_directory_contents(val_path)


Checking training directories:
Found 11 images in: /content/drive/MyDrive/Dataset/Train/Normal/Front
Found 11 images in: /content/drive/MyDrive/Dataset/Train/Normal/Back
Found 11 images in: /content/drive/MyDrive/Dataset/Train/Normal/Nailbed
Found 76 images in: /content/drive/MyDrive/Dataset/Train/Controlled/Front
Found 64 images in: /content/drive/MyDrive/Dataset/Train/Controlled/Back
Found 38 images in: /content/drive/MyDrive/Dataset/Train/Controlled/Nailbed
Found 97 images in: /content/drive/MyDrive/Dataset/Train/DFC/Front
Found 85 images in: /content/drive/MyDrive/Dataset/Train/DFC/Back
Found 63 images in: /content/drive/MyDrive/Dataset/Train/DFC/Nailbed

Checking validation directories:
Found 3 images in: /content/drive/MyDrive/Dataset/Validation/Normal/Front
Found 3 images in: /content/drive/MyDrive/Dataset/Validation/Normal/Back
Found 3 images in: /content/drive/MyDrive/Dataset/Validation/Normal/Nailbed
Found 7 images in: /content/drive/MyDrive/Dataset/Validation/Controlled/Fron

In [ ]:
def create_generators(category, orientation):
    train_dir = os.path.join(train_path, category, orientation)
    val_dir = os.path.join(val_path, category, orientation)

    train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=image_size,
        batch_size=batch_size,
        classes=[category],  # Specify the class based on the folder
        class_mode='categorical',  # Ensure categorical mode for one-hot encoding
        shuffle=True
    )

    validation_generator = val_datagen.flow_from_directory(
        val_dir,
        target_size=image_size,
        batch_size=batch_size,
        classes=[category],  # Specify the class based on the folder
        class_mode='categorical',  # Ensure categorical mode for one-hot encoding
        shuffle=False
    )

    return train_generator, validation_generator



In [ ]:
def build_model():
    model = models.Sequential([
        layers.Input(shape=(224, 224, 3)),
        tf.keras.applications.EfficientNetB0(weights='imagenet', include_top=False, pooling='avg'),
        layers.Dense(3, activation='softmax')  # 3 classes: Normal, Controlled, Severe
    ])

    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    return model


In [ ]:
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define paths
train_path = '/content/drive/MyDrive/Dataset/Train'
val_path = '/content/drive/MyDrive/Dataset/Validation'

# Define image data generators
batch_size = 32
image_size = (224, 224)

train_datagen = ImageDataGenerator(rescale=1.0/255.0)
val_datagen = ImageDataGenerator(rescale=1.0/255.0)

# Define orientations
orientations = ['Front', 'Back', 'Nailbed']
categories = ['Normal', 'Controlled', 'Severe']

# Function to create data generators
def create_generators(orientation):
    train_dir = os.path.join(train_path, orientation)  # Changed here
    val_dir = os.path.join(val_path, orientation)      # Changed here

    print(f"Training Directory: {train_dir} - Images: {len(os.listdir(train_dir)) if os.path.exists(train_dir) else 'Directory does not exist'}")
    print(f"Validation Directory: {val_dir} - Images: {len(os.listdir(val_dir)) if os.path.exists(val_dir) else 'Directory does not exist'}")

    train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=image_size,
        batch_size=batch_size,
        class_mode='categorical',  # For multi-class classification
        shuffle=True
    )

    validation_generator = val_datagen.flow_from_directory(
        val_dir,
        target_size=image_size,
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=False
    )

    # Check if classes are recognized
    print(f"Classes found in training generator: {train_generator.class_indices}")
    print(f"Classes found in validation generator: {validation_generator.class_indices}")

    return train_generator, validation_generator

# Build and train models for each orientation
for orientation in orientations:
    print(f"\nTraining model for orientation: {orientation}")
    train_gen, val_gen = create_generators(orientation)

    # Build and train the model
    model = build_model()  # Assuming build_model is defined elsewhere
    history = model.fit(
        train_gen,
        validation_data=val_gen,
        epochs=10  # Adjust epochs as needed
    )




Training model for Normal - Front orientation
Training Directory: /content/drive/MyDrive/Dataset/Train/Normal/Front - Images: 11
Validation Directory: /content/drive/MyDrive/Dataset/Validation/Normal/Front - Images: 3
Found 33 images belonging to 1 classes.
Found 9 images belonging to 1 classes.
Classes found in training generator: {'Normal': 0}
Classes found in validation generator: {'Normal': 0}
